In [1]:
import geopy
import pandas as pd

In [2]:
couples_df = pd.read_csv('couples/20140319/couples_FLA_20140319.csv', sep='\t')

In [7]:
couples_df.columns

Index(['county_code_L', 'voter_id_L', 'last_name_L', 'first_name_L',
       'residence_addr_line_1_L', 'residence_addr_line_2_L',
       'residence_city_L', 'residence_zipcode_L', 'gender_L', 'race_L',
       'birth_date_L', 'registration_date_L', 'party_affiliation_L',
       'precinct_L', 'voter_status_L', 'age_L', 'residence_zipcode_5_L',
       'residence_zipcode_4_L', 'uniq_addr', 'county_code_R', 'voter_id_R',
       'last_name_R', 'first_name_R', 'residence_addr_line_1_R',
       'residence_addr_line_2_R', 'residence_city_R', 'residence_zipcode_R',
       'gender_R', 'race_R', 'birth_date_R', 'registration_date_R',
       'party_affiliation_R', 'precinct_R', 'voter_status_R', 'age_R',
       'residence_zipcode_5_R', 'residence_zipcode_4_R', 'age_diff'],
      dtype='object')

In [5]:
couples_df['uniq_addr'] = couples_df[['residence_addr_line_1_L', 'residence_city_L']].apply(lambda x: ' '.join(x), axis=1)

In [9]:
couples_df['residence_zipcode_5'] = couples_df['residence_zipcode_L'].apply(lambda x: int(str(x)[:5]))
couples_df['uniq_addr_2'] = couples_df[['residence_addr_line_1_L', 'residence_addr_line_2_L', 'residence_city_L', 'residence_zipcode_5']].apply(lambda x: ' '.join([str(y) for y in x]), axis=1)
import re
couples_df['uniq_addr_2'] = couples_df['uniq_addr_2'].apply(lambda x: re.sub("[^0-9a-zA-Z\s]+", '', x))
couples_df['uniq_addr_2'] = couples_df['uniq_addr_2'].apply(lambda x: x.strip())


In [16]:
fla_lat = pd.read_csv("geolocation.csv" , sep='\t')

In [19]:
fla_lat.columns = ["uniq_addr", "addr", "lat","long","pt"]

In [14]:
cdf = couples_df[["uniq_addr", "uniq_addr_2"]]

In [20]:
fla_lat = pd.merge(cdf, fla_lat, on="uniq_addr",)

In [22]:
fla_lat.to_csv("geolocation_latest.csv", sep="\t")

In [ ]:
addr_to_zip = {couples_df.uniq_addr[i]: couples_df.residence_zipcode_5_L[i] for i in range(len(couples_df))}

In [66]:
uniq_addresses = couples_df['uniq_addr'].unique()

In [67]:
len(uniq_addresses)

20009

In [68]:
uniq_addresses

array(['609   yorkshire dr flagler beach', '123   cochise ct palm coast',
       '18   san miguel ct palm coast', ...,
       '128   parkview dr palm coast', '58   red mill dr palm coast',
       '53   ocean oaks ln palm coast'], dtype=object)

In [63]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="pp")
from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
location = geocode({ 'postalcode': '32164'})


In [64]:
location

Location(Palm Coast, Flagler County, Florida, 32164, United States of America, (29.4988703677898, -81.2246984147163, 0.0))

In [ ]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import tqdm
geolocator = Nominatim(user_agent="pp")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=0.8)

failed = []
with open('geolocation.csv', 'w') as file:
    for addr in tqdm.tqdm(uniq_addresses):
#         if i % 100 == 0:
#             print('Iteration: ' + str(i))
        try:
            location = geocode(addr)
        except exception as e:
            location = None
        if location:
            write_line = '\t'.join([addr, str(location.address), str(location.latitude), str(location.longitude), str(location.altitude)]) + '\n'
            file.write(write_line)
        else:
            try:
                location = geocode({'postalcode': addr_to_zip[addr]})
            except Exception as e:
                location = None
            if location:
                write_line = '\t'.join([addr, str(location.address), str(location.latitude), str(location.longitude), str(location.altitude)]) + '\n'
                file.write(write_line)
            else:
                failed.append(addr)
#                 print('Failed: ' + str(len(failed)))

Iteration: 0
Iteration: 100
